In [2]:
import os
import glob
import sys
import subprocess
import xarray as xr
import numpy as np
import pandas as pd
from time import perf_counter

THIS_DIR = os.path.abspath('')
sys.path.append(os.path.join(THIS_DIR, os.pardir, os.pardir, os.pardir, os.pardir))


from src.postprocessing.py.qnc_defintions import Output_format
from src.postprocessing.py.qnc_output_parser import QNC_output_parser
from src.postprocessing.py.qnc_ncdf_reader import QNC_ncdf_reader


In [3]:
OUTPUT_DIR = "/Net/Groups/BSI/scratch/ppapastefanou/simulations/QPy/jsbach_spq/17_transient_slurm_array/output/2701"

In [4]:
parser = QNC_output_parser(OUTPUT_DIR)
parser.Read()
output = parser.Available_outputs['fluxnetdata']
nc_output = QNC_ncdf_reader(OUTPUT_DIR,
                                        output.Categories,
                                        output.Identifier,
                                        output.Time_resolution
                                        )

nc_output.Parse_env_and_variables()
nc_output.Read_all_1D()
nc_output.Close()

RTOBSPATH = "/Net/Groups/BSI/work_scratch/ppapastefanou/data/Fluxnet_detail/eval_processed"

df_psi_stem = pd.read_csv(os.path.join(RTOBSPATH,"PsiStem2023.csv"))
df_psi_stem['date'] = pd.to_datetime(df_psi_stem['date'])

df_mod_hyd = nc_output.Datasets_1D['PHYD']
df_merge = pd.merge(df_mod_hyd, df_psi_stem, on='date', how='inner')   
rmse = np.sqrt(((df_merge['psi_stem_avg'] - df_merge['FAG']) ** 2).mean())
print(rmse)

0.09829808090083333


0.09829808090083333
